In [0]:
import re
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import matplotlib.pyplot as plt

> **Data from 20150101 to today**



In [0]:
from datetime import datetime
start = '20150101'
today = datetime.today().strftime('%Y%m%d')

## Litecoin

In [0]:
url = 'https://coinmarketcap.com/currencies/litecoin/historical-data/?start='+start+'&end='+today
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
divs = soup.find_all(class_ = 'table')

In [0]:
csvfile = open('litecoin_data.csv', 'w')
header = ['Open','High','Low','Close', 'Volume', 'Market Cap'] # not written to csv
filewriter = csv.writer(csvfile)
filewriter.writerow(header)

In [0]:
entries = soup.find_all('td')
temp = []
for i in range(len(entries)):
  
  if i % 7 == 0:
    if i != 0:
      filewriter.writerow(temp)
      temp = []
  else:
    temp.append(entries[i].text)
    temp = [temp[i].replace(',', '') for i in range(len(temp))]
    

In [0]:
df_1 = pd.read_csv('./litecoin_data.csv')
df_1.apply(pd.to_numeric)

## Ethereum

In [0]:
url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start='+start+'&end='+today
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
divs = soup.find_all(class_ = 'table')

In [0]:
csvfile = open('ethereum_data.csv', 'w')
header = ['Open','High','Low','Close', 'Volume', 'Market Cap'] # not written to csv
filewriter = csv.writer(csvfile)
filewriter.writerow(header)

39

In [0]:
entries = soup.find_all('td')
temp = []
for i in range(len(entries)):
  
  if i % 7 == 0:
    if i != 0:
      filewriter.writerow(temp)
      temp = []
  else:
    temp.append(entries[i].text)
    temp = [temp[i].replace(',', '') for i in range(len(temp))]
    

In [0]:
df_2 = pd.read_csv('./ethereum_data.csv')
df_2.apply(pd.to_numeric)

In [0]:
corr = df_2.corr()
corr.style.background_gradient(cmap='coolwarm')

,Open,High,Low,Close,Volume,Market Cap
Open,1,0.998583,0.996535,0.99652,0.372037,0.996064
High,0.998583,1,0.996569,0.998488,0.376361,0.997646
Low,0.996535,0.996569,1,0.998004,0.367873,0.997879
Close,0.99652,0.998488,0.998004,1,0.373292,0.999377
Volume,0.372037,0.376361,0.367873,0.373292,1,0.394669
Market Cap,0.996064,0.997646,0.997879,0.999377,0.394669,1


## Bitcoin

### From blockchain.com (12 features, manually download file)

Imported data from https://www.blockchain.com/en/charts

All csv files should have 729 rows.

In [0]:
col1 = pd.read_csv('./avg-block-size.csv')
col2 = pd.read_csv('./blocks-size.csv')
col3 = pd.read_csv('./cost-per-transaction-percent.csv')
col4 = pd.read_csv('./hash-rate.csv')
col5 = pd.read_csv('./market-cap.csv')
col6 = pd.read_csv('./market-price.csv')
col7 = pd.read_csv('./median-confirmation-time.csv')
col8 = pd.read_csv('./n-transactions-per-block.csv')
col9 = pd.read_csv('./n-transactions.csv')
col10 = pd.read_csv('./total-bitcoins.csv')
col11 = pd.read_csv('./trade-volume.csv')
col12 = pd.read_csv('./transaction-fees.csv')

In [0]:
temp = col1.iloc[:,1].values


In [0]:
csvfile = open('bitcoin_1.csv', 'w')
header = ['avg-block-size','blocks-size','cost-per-transaction-percent',
          'hash-rate','market-cap', 'market-price','median-confirmation-time',
          'n-transactions-per-block','n-transactions','total-bitcoins',
          'trade-volume','transaction-fees'] # not written to csv
filewriter = csv.writer(csvfile)
filewriter.writerow(header)

201

In [0]:
for i in range(0, col1.shape[0]):
  temp = []
  temp.append(col1.iloc[:,1].values[i])
  temp.append(col2.iloc[:,1].values[i])
  temp.append(col3.iloc[:,1].values[i])
  temp.append(col4.iloc[:,1].values[i])
  temp.append(col5.iloc[:,1].values[i])
  temp.append(col6.iloc[:,1].values[i])
  temp.append(col7.iloc[:,1].values[i])
  temp.append(col8.iloc[:,1].values[i])
  temp.append(col9.iloc[:,1].values[i])
  temp.append(col10.iloc[:,1].values[i])
  temp.append(col11.iloc[:,1].values[i])
  temp.append(col12.iloc[:,1].values[i])
  
  filewriter.writerow(temp)

In [0]:
import matplotlib.pyplot as plt

df_3 = pd.read_csv('./bitcoin_1.csv')
# plt.matshow(df.corr())
# plt.show()
corr = df_3.corr()
corr.style.background_gradient(cmap='coolwarm')

,avg-block-size,blocks-size,cost-per-transaction-percent,hash-rate,market-cap,market-price,median-confirmation-time,n-transactions-per-block,n-transactions,total-bitcoins,trade-volume,transaction-fees
avg-block-size,1,0.310749,-0.513581,0.048734,0.0529786,0.0434974,0.302069,0.788155,0.684007,0.300616,0.105988,0.263694
blocks-size,0.310749,1,0.28253,0.898586,-0.219007,-0.254,-0.331422,0.395001,0.312081,0.999067,-0.345543,-0.463334
cost-per-transaction-percent,-0.513581,0.28253,1,0.452855,0.0329008,0.0198587,-0.415665,-0.41547,-0.364987,0.287935,-0.245897,-0.340876
hash-rate,0.048734,0.898586,0.452855,1,-0.176585,-0.211189,-0.440587,0.116959,0.0940413,0.907117,-0.364607,-0.553539
market-cap,0.0529786,-0.219007,0.0329008,-0.176585,1,0.999162,0.0612417,-0.0670312,0.0682875,-0.23739,0.676308,0.563529
market-price,0.0434974,-0.254,0.0198587,-0.211189,0.999162,1,0.0734224,-0.0771316,0.0601132,-0.272123,0.687128,0.578753
median-confirmation-time,0.302069,-0.331422,-0.415665,-0.440587,0.0612417,0.0734224,1,0.278211,0.0759837,-0.333585,0.18591,0.353231
n-transactions-per-block,0.788155,0.395001,-0.41547,0.116959,-0.0670312,-0.0771316,0.278211,1,0.878587,0.393215,0.0234037,0.296185
n-transactions,0.684007,0.312081,-0.364987,0.0940413,0.0682875,0.0601132,0.0759837,0.878587,1,0.306267,0.114178,0.379606
total-bitcoins,0.300616,0.999067,0.287935,0.907117,-0.23739,-0.272123,-0.333585,0.393215,0.306267,1,-0.357554,-0.473507


Correlation to **market-price**:


*   blocks-size: -0.254
*   hash-rate: -0.211
*   market-cap: 0.999
*   total-bitcoins: -0.272
*   trade-volume: **0.687**
*   transaction-fees: **0.579**



### From bitcoin.com (39 features)

https://charts.bitcoin.com/btc/chart/price

Using Selenium without opening the browser

Download 39 csv files and merge to one

Delete data before 2015

#### Selenium on Colab (not working, cannot download files)

In [0]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# import time
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# prefs = {"download.default_directory" : "/content/drive/My\ Drive/"}
# chrome_options.add_experimental_option("prefs",prefs)

In [0]:
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# driver.get('https://charts.bitcoin.com/btc/chart/price')
# time.sleep(10)
# driver.find_element_by_id("sidebar-tools-btn").click()
# time.sleep(5)
# driver.find_element_by_id("data-download").click()


#### Selenium in .py (work), data available starting after 2010/8/17

**7/22: Merge to one csv**

start from 2013/03/01

todo: combine to one csv, correlation matrix

In [2]:
import csv
import os
from datetime import datetime as dt
!pip install xlsxwriter
import numpy as np
import pandas as pd

     |████████████████████████████████| 143kB 4.9MB/s 


In [0]:
import xlsxwriter

output = xlsxwriter.Workbook('bitcoin_merged.xlsx')
filewriter = output.add_worksheet()

directory = "../content"
col_cnt = 1

for filename in os.listdir(directory):
  if filename.endswith(".csv"):
    df = pd.read_csv(filename)
    if col_cnt == 1: 
      filewriter.write(0, 0, df.columns[0]) # write header
    filewriter.write(0, col_cnt, df.columns[1]) # write header

    row_cnt = 1
    start_date = dt.strptime("10/08/17", "%y/%m/%d")
    
    for i in range(df.shape[0]):
      current_date = dt.strptime(df.iloc[i][0], "%Y-%m-%d")
      if start_date <= current_date:
        if(col_cnt == 1):
          filewriter.write(row_cnt, 0, df.iloc[i][0])
        filewriter.write(row_cnt, col_cnt, df.iloc[i][1])
        row_cnt += 1

    col_cnt += 1

        
output.close()

In [30]:
import matplotlib.pyplot as plt

bitcoin_df = pd.read_excel('./bitcoin_merged.xlsx')
# plt.matshow(df.corr())
# plt.show()
corr = bitcoin_df.corr()
corr.style.background_gradient(cmap='coolwarm')